In [1]:
import numpy as np
import struct
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline
from tqdm import tqdm

## Read dynamic list-mode CASToR datafile

In [2]:
datafile='data/poumon_pat6_list-mode_0_tof_ref0_1_sur100.cdf'

In [3]:
# Structure of datafile for 32 bit list-mode ( including TOF )
struct.calcsize('IfffffII')


# Structure of compressed datafile for 24 bit list-mode ( including TOF )
# uint32_t : time tag
# float: Normalization + Attenuation
# float: Scatter + randoms
# float: TOF arrival time diff
# uint32_t: CrystalID1
# uint32_t: CrystalID2
#struct.calcsize('IfffII')

32

In [4]:
# Get last time tag
with open(datafile, mode='rb') as file: # b is important -> binary
    file.seek(os.path.getsize(datafile)-32)
    # Read last phrase
    datafilePack = file.read(32)
    # read its time tag , as the last time tag
    lastTimeTag = int(struct.unpack("I",datafilePack[:4])[0])

In [5]:
lastTimeTag

3599999

## Define Framing for the study ( on study refference time == injection time )

In [6]:
frm="0,10,20,30,40,50,60,70,80,90,100, \
110,120,140,160,180,200,220,240,260,280,300,320,340,   \
360,420,480,540,600,720,840,960,1080,1200,1500,1800,2100,2400,2700,3000,3300:300"

In [7]:
frames_phrases=frm.split(",")
FrameStart=np.zeros([len(frames_phrases)],dtype='int')
FrameDuration=np.zeros([len(frames_phrases)],dtype='int')

fr=0
for phrase in frames_phrases:
    #print(phrase)
    if (phrase.find(":") != -1) :
        FrameStart[fr]=float(phrase.split(":")[0])
        FrameDuration[fr]=float(phrase.split(":")[1])
    else:
        FrameStart[fr]=float(phrase)
    fr+=1

# Loop over farmes duration and replace zeros with time until next frame
for idx,val in enumerate(FrameDuration):
    if val==0:
        FrameDuration[idx]=FrameStart[idx+1]-FrameStart[idx]

In [8]:
FrameStart.shape

(41,)

## Read respiratory triggers time tags

In [9]:
resp_triggers=pd.read_csv('data/LIST0000.resp')
resp_triggers.columns = ['trigger_time_tag']

In [10]:
# The data were unlisted from 60 seconds onwards ( and refference is 60sec)
# We need to apply the same offset to the respiratory gates
resp_triggers = (resp_triggers-60000)
# Then remove triggers taking place before injection time (time=0)
resp_triggers = resp_triggers[resp_triggers['trigger_time_tag'] > 0]

In [11]:
resp_triggers.diff().mean()

trigger_time_tag    3419.779258
dtype: float64

In [12]:
resp_triggers = np.array(resp_triggers.trigger_time_tag.tolist())
resp_triggers_diff = np.diff(resp_triggers)

In [13]:
# Statistics of resp tags
print("Number of respiratory tags: %d"%resp_triggers.shape[0])
print("Mean respiratory rate: %f ms"%np.diff(resp_triggers).mean())
accepted_count=((np.diff(resp_triggers)<4000*1.66) & (np.diff(resp_triggers)>4000*0.33)).sum()
print("Number of cycles to include: %d "%accepted_count)
print("Rejection fraction: {0:.4f}%".format((1-accepted_count/resp_triggers.shape[0])*100))

Number of respiratory tags: 1052
Mean respiratory rate: 3419.779258 ms
Number of cycles to include: 952 
Rejection fraction: 9.5057%


In [14]:
# Check if last respiratory trigger takes place before end of file
print (resp_triggers[-1]<lastTimeTag)

True


## Create time-tag info array for gates phase, gate index and rejected triger phases

### In this step we create an information matrix for each time-tag within the list mode file.
In each time tag we have the following information:
#0: time-tag
#1: respiratory cycle index
#2: Frame index
#3: Accepted or rejected  based on respiratory cycle duration ( within 30% of 4000ms ) 
#4: Use for Q.Static ?  (Yes=1 or No=0)
#5: Q.Freeze (Gate) index number

In [15]:
InfoMatrix = np.zeros([lastTimeTag+1,6],dtype=int)

In [16]:
InfoMatrix[:,0] = np.arange(lastTimeTag+1)

In [68]:
# Set respiratory cycle index and check respiratory cycle duration
current_resp_cycle=0
current_frame=0
# Reject all tags before the first respiratory trigger and after the last 
for tag in range(resp_triggers[0]):
    InfoMatrix[tag,3]=0

for tag in range(resp_triggers[-1],lastTimeTag):
    InfoMatrix[tag,3]=0

for tag in tqdm(range(resp_triggers[0],resp_triggers[-1])):
    # Check if we need to update frame index
    if tag>=((FrameStart[current_frame]+FrameDuration[current_frame])*1000):
        current_frame+=1;
    InfoMatrix[tag,2]=current_frame
    
    # it tag is greater or equal to the following resp trigger, move one cycle up
    if tag>=resp_triggers[current_resp_cycle+1]:
        current_resp_cycle+=1;
    # Set the current_resp_cycle index
    InfoMatrix[tag,1]=current_resp_cycle
    
    # Check duration of cycle to reject or keep
    if (4000*0.33<resp_triggers_diff[current_resp_cycle]<4000*1.66):
        InfoMatrix[tag,3]=1
    
    # Trigger is already placed 30% forward from the peak ( by GE )
    # we keep the first 50% of the data in the resp cycle for Q.Static data
    if (tag<resp_triggers[current_resp_cycle]+resp_triggers_diff[current_resp_cycle]/2):
        InfoMatrix[tag,4]=1
      
    # Trigger is placed 30% forward from the peak ( by GE )
    # we splitt the data in the resp cycle in 5 parts/gates for Q.Static data    
    for gate in range(1,6):
        gate_start=resp_triggers[current_resp_cycle]+(resp_triggers_diff[current_resp_cycle]/5.)*(gate-1)
        gate_end=resp_triggers[current_resp_cycle]+(resp_triggers_diff[current_resp_cycle]/5.)*(gate)
        if (gate_start<=tag<gate_end):
            InfoMatrix[tag,5]=gate-1


100%|███████████████████████████████████████████████████████████████████████████████████████████| 3594188/3594188 [03:46<00:00, 15889.50it/s]


In [25]:
np.savetxt('data/InfoMatrix.txt',InfoMatrix.astype(int),fmt='%i', delimiter=",")

In [69]:
InfoMatrix.astype(np.int64).tofile('data/InfoMatrix.bin')

In [44]:
InfoMatrix = np.loadtxt('data/InfoMatrix.txt', delimiter=",")

In [24]:
#InfoMatrix[1495:1555]
InfoMatrix[3509999,:]

array([3509999,    1023,      40,       1,       1,       1])

In [28]:
np.histogram(InfoMatrix[:,5],10)

(array([  5812,      0, 718851,      0, 718848,      0, 718848,      0,
        718848, 718793], dtype=int64),
 array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ]))

In [33]:
np.sum(InfoMatrix[:,5]>5)

0

## Process the list-mode file and create Q.Static Q.Freeze list files (with frames)

In [ ]:
# Q.Static
pbar = tqdm(total=int(os.path.getsize(datafile)/24))   
    
with open(datafile, mode='rb') as file:
    # Create Q.Static file
    QStaticFile = open(os.path.basename(datafile).split('.')[0]+'_QStatic.cdf', "wb")
    
    # Read first datafilePack
    datafilePack = file.read(24)
    count=1
    QStaticCount=0
    # Loop over all data untill end of file
    while datafilePack:
        # get time tag
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        # Check if we will use this time tag
        if InfoMatrix[timetag,2]==1:
            count+=1
            # Check if we will use this time tag for Q.Static
            if InfoMatrix[timetag,3]==1:
                QStaticCount+=1
                QStaticFile.write(datafilePack)

        # Read next datafilePack
        datafilePack = file.read(24)
        #print progress
        pbar.update(int(count))

    # close output files
    QStaticFile.close()
    # close progress bar 
    pbar.close()

In [ ]:
print("Number of events in Q.Static CASToR Datafile: %d"%QStaticCount)
print("Number of total events (accepted) from original CASToR Datafile: %d"%count)

In [ ]:
InfoMatrix[2614]

In [ ]:
with open(datafile, mode='rb') as file:
    datafilePack = file.read(24)
    while datafilePack:
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        if (timetag==2615):
            print("found")
            break
        datafilePack = file.read(24)

In [ ]:
datafilePack

In [ ]:
struct.unpack('IfffII', datafilePack)

In [ ]:
# Q.Freeze
pbar = tqdm(total=int(os.path.getsize(datafile)/24))   
    
with open(datafile, mode='rb') as file:
    # Create Q.Static file
    
    QFreezeFile_gate1 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate1.cdf', "wb")
    QFreezeFile_gate2 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate2.cdf', "wb")
    QFreezeFile_gate3 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate3.cdf', "wb")
    QFreezeFile_gate4 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate4.cdf', "wb")
    QFreezeFile_gate5 = open(os.path.basename(datafile).split('.')[0]+'_QFreeze_Gate5.cdf', "wb")
    QFreezeFiles=[QFreezeFile_gate1,QFreezeFile_gate2,QFreezeFile_gate3,QFreezeFile_gate4,QFreezeFile_gate5]
    
    # Read first datafilePack
    datafilePack = file.read(24)
    count=1
    GateCount=np.zeros([5])
    # Loop over all data untill end of file
    while datafilePack:
        # get time tag
        timetag = int(struct.unpack("I",datafilePack[:4])[0])
        # Check if we will use this time tag
        if InfoMatrix[timetag,2]==1:
            count+=1
            # Check to which gate this event bellongs to
            gate =  int(InfoMatrix[timetag,4])
            QFreezeFiles[gate-1].write(datafilePack)
            GateCount[gate-1]+=1


        # Read next datafilePack
        datafilePack = file.read(24)
        #print progress
        pbar.update(int(count))

    # close output files
    QStaticFile.close()
    # close progress bar 
    pbar.close()

In [ ]:
QFreezeFiles[1-1]

## Calculate quantification factors

In [36]:
head_curve=pd.read_csv('data/prompts_headcurve.hc',sep=',',header=None)
head_curve.columns=['time','prompts']
prompt_array = np.array(head_curve.prompts)

In [37]:
prompt_array = np.array(head_curve.prompts)

In [38]:
prompt_array.shape

(3599999,)

In [37]:
prompt_array[tag]

395

In [40]:
QStatic_counts_per_frame=np.zeros([np.size(FrameStart),2],dtype='int')
QStatic_mseconds_per_frame=np.zeros([np.size(FrameStart),2],dtype='int')

for frm in range(np.size(FrameStart)):
    for tag in range(FrameStart[frm]*1000,FrameStart[frm]*1000+FrameDuration[frm]*1000):
        if ((InfoMatrix[tag,3]==1)and(InfoMatrix[tag,4]==0)):
            QStatic_counts_per_frame[frm,0]+=prompt_array[tag]
            QStatic_mseconds_per_frame[frm,0]+=1
        if ((InfoMatrix[tag,3]==1)and(InfoMatrix[tag,4]==1)):
            QStatic_counts_per_frame[frm,1]+=prompt_array[tag]
            QStatic_mseconds_per_frame[frm,1]+=1
            

In [41]:
for frm in range(np.size(FrameStart)):
    print(str(QStatic_counts_per_frame[frm,0])+","+str(QStatic_counts_per_frame[frm,1]))

1166160,2750263
20614616,19043671
12651380,12604682
9413227,9752994
9956982,9674381
8626578,10211739
9660610,8830682
8866733,9813369
9405236,9285143
9984158,8652089
8959972,9549498
8379555,9884698
18077914,17564165
18503548,16746187
17037149,17749543
17420887,17017694
16911471,17225178
17115181,16633524
18070273,15527830
15583987,17696646
17485022,15522162
16918205,15865530
15488599,17093960
17246648,15221329
47836251,48582294
47110290,47135635
46197464,46051818
44917554,45772584
88243432,88788724
85567302,85218879
55664000,56569214
79731044,78236056
51886130,52351610
150885945,152017503
152272124,152200848
134484245,135956408
143971579,144103322
132345696,132589655
125327939,125882632
117426625,117126683
113047144,113470089


In [43]:
for frm in range(np.size(FrameStart)):
    print(str(QStatic_mseconds_per_frame[frm,0]/1000)+","+str(QStatic_mseconds_per_frame[frm,1]/1000))

3.24,4.146
5.186,4.754
5.15,4.85
4.9,5.1
5.07,4.87
4.6,5.4
5.173,4.707
4.77,5.23
5.05,4.95
5.369,4.631
4.858,5.142
4.579,5.421
10.012,9.928
10.398,9.602
9.68,10.32
10.001,9.999
9.809,10.191
10.02,9.98
10.67,9.33
9.27,10.73
10.49,9.51
10.219,9.781
9.441,10.559
10.56,9.44
29.797,30.082
30.002,29.998
30.026,29.974
29.664,30.276
59.72,60.16
60.14,59.8
40.421,41.0
60.56,59.26
40.458,40.483
124.303,124.757
136.0,135.48
125.845,126.856
146.337,146.064
145.54,145.32
145.129,145.271
143.857,143.01
146.525,146.53


In [70]:
Qfreeze_counts_per_frame=np.zeros([np.size(FrameStart),5],dtype='int')
Qfreeze_mseconds_per_frame=np.zeros([np.size(FrameStart),5],dtype='int')

for frm in range(np.size(FrameStart)):
    for tag in range(FrameStart[frm]*1000,FrameStart[frm]*1000+FrameDuration[frm]*1000):
        if ((InfoMatrix[tag,3]==1)):
            Qfreeze_counts_per_frame[frm,InfoMatrix[tag,5]-1]+=prompt_array[tag]
            Qfreeze_mseconds_per_frame[frm,InfoMatrix[tag,5]-1]+=1


In [71]:
for frm in range(np.size(FrameStart)):
    print(str(Qfreeze_counts_per_frame[frm,0])+","+str(Qfreeze_counts_per_frame[frm,1])\
          +","+str(Qfreeze_counts_per_frame[frm,2])\
          +","+str(Qfreeze_counts_per_frame[frm,3])\
          +","+str(Qfreeze_counts_per_frame[frm,4]))

611140,451397,448927,492102,1912857
8705094,9182209,8970638,7094207,5706139
4964698,4753452,4532005,5771967,5233940
3908141,3910384,3923347,3535192,3889157
3869973,3841829,3814659,4226098,3878804
4087019,4058178,3617183,2986027,4089910
3146461,3124608,3554408,4548055,4117760
4187449,3562401,3538148,3552367,3839737
3578435,3779410,3751643,3766968,3813923
3636120,3996013,3986907,4001069,3016138
3917101,3897748,3879778,3136294,3678549
3945796,3964323,2984822,3416266,3953046
7061280,6335454,7275100,7638549,7331696
6611559,7671946,7645844,7022164,6298222
7081972,7246662,6676009,6732360,7049689
6402054,6263734,6819198,7464854,7488741
7239687,7731825,6571116,6480800,6113221
6851711,5887044,7022729,7144769,6842452
5994737,7243687,7221102,7231551,5907026
7087474,7213198,5985167,5990068,7004726
5883221,6018408,7212003,7265631,6627921
6640018,6765147,6757822,6772707,5848041
6899756,6968407,6402272,5606776,6705348
6081462,6190559,6697130,7459526,6039300
19231129,19167147,19073095,19196687,19750487

In [73]:
#Qfreeze_mseconds_per_frame=Qfreeze_mseconds_per_frame/1000.;
for frm in range(np.size(FrameStart)):
    print(str(Qfreeze_mseconds_per_frame[frm,0])+","+str(Qfreeze_mseconds_per_frame[frm,1])\
          +","+str(Qfreeze_mseconds_per_frame[frm,2])\
          +","+str(Qfreeze_mseconds_per_frame[frm,3])\
          +","+str(Qfreeze_mseconds_per_frame[frm,4]))

1.35,1.296,1.296,1.296,2.148
2.21,2.264,2.264,1.79,1.412
1.94,1.94,1.94,2.24,1.94
2.04,2.04,2.04,1.84,2.04
1.948,1.948,1.948,2.148,1.948
2.16,2.16,1.931,1.589,2.16
1.68,1.68,1.909,2.424,2.187
2.227,1.908,1.908,1.908,2.049
1.904,2.02,2.02,2.02,2.036
1.945,2.148,2.148,2.147,1.612
2.108,2.108,2.108,1.696,1.98
2.168,2.168,1.635,1.86,2.169
4.0,3.504,4.037,4.223,4.176
3.812,4.308,4.308,3.936,3.636
4.128,4.128,3.801,3.815,4.128
3.773,3.596,3.923,4.28,4.428
4.303,4.48,3.817,3.752,3.648
4.122,3.452,4.115,4.179,4.132
3.602,4.272,4.272,4.262,3.592
4.292,4.292,3.566,3.558,4.292
3.608,3.608,4.334,4.352,4.098
4.088,4.088,4.088,4.087,3.649
4.244,4.244,3.911,3.408,4.193
3.776,3.776,4.109,4.563,3.776
11.9,11.9,11.9,11.947,12.232
12.132,12.132,12.132,11.804,11.8
11.88,11.88,11.88,12.206,12.154
12.22,12.22,12.011,11.543,11.946
24.064,24.064,23.641,24.047,24.064
23.92,23.92,24.282,23.898,23.92
16.4,16.176,16.002,16.443,16.4
23.704,23.928,24.372,24.112,23.704
16.08,16.08,16.08,16.338,16.363
50.016,50.016,4

In [63]:
InfoMatrix[3306615,:]

array([3306615,     959,      40,       1,       0,       5])

In [64]:
InfoMatrix[3306614,:]

array([3306614,     959,      40,       1,       0,       4])